In [4]:
import pandas as pd
from datetime import datetime, timedelta
import folium
import time
from IPython.display import display, clear_output

# Load GTFS data
data_path = r"C:\Users\TULPAR\JupyterLab Projects\GTFS Project\dataset"
"C:\Users\TULPAR\JupyterLab Projects\GTFS Project"
stops = pd.read_csv(f"{data_path}/stops.txt")
routes = pd.read_csv(f"{data_path}/routes.txt")
trips = pd.read_csv(f"{data_path}/trips.txt")
stop_times = pd.read_csv(f"{data_path}/stop_times.txt")
calendar = pd.read_csv(f"{data_path}/calendar.txt")
shapes = pd.read_csv(f"{data_path}/shapes.txt")

# Function to convert times
def convert_time(time_str):
    try:
        # Split the time string
        parts = time_str.split(':')
        hours, minutes, seconds = int(parts[0]), int(parts[1]), int(parts[2])
        
        # If the hour is 24 or more, adjust the time
        if hours >= 24:
            hours -= 24
        # Return the adjusted time in HH:MM:SS format
        return f"{hours:02}:{minutes:02}:{seconds:02}"
    except ValueError as e:
        print(f"Error converting time: {time_str} - {e}")
        return None

# Apply the conversion to arrival_time and departure_time

stop_times['arrival_time'] = stop_times['arrival_time'].apply(convert_time)
stop_times['departure_time'] = stop_times['departure_time'].apply(convert_time)

# Drop rows with None values after conversion
stop_times = stop_times.dropna(subset=['arrival_time', 'departure_time'])

# Save the modified stop_times to a new file
stop_times.to_csv('stop_times_modified.txt', index=False)

print("Stops data:")
display(stops.head())

print("\nRoutes data:")
display(routes.head())

print("\nTrips data:")
display(trips.head())

print("\nStop Times data:")
display(stop_times)

print("\nCalendar data:")
display(calendar.head())

print("\nShapes data:")
display(shapes.head())

# Simulation parameters
#start_time = datetime.strptime('06:00:00', '%H:%M:%S')  # Simulation start time
now_time=datetime.now()
current_time=datetime.strptime(now_time.strftime('%H:%M:%S'),'%H:%M:%S')
print(current_time)
start_time=current_time

end_time = datetime.strptime('23:00:00', '%H:%M:%S')    # Simulation end time

print("Current time is: ", current_time)
time_step = timedelta(seconds=30)  # Simulation time step

# Initialize the map centered on the city
city_latitude = stops['stop_lat'].mean()
city_longitude = stops['stop_lon'].mean()
map = folium.Map(location=[city_latitude, city_longitude], zoom_start=12)

# Initialize buses
buses = [{
    'trip_id': trip.trip_id,
    'route_id': trip.route_id,
    'current_stop_sequence': 0,
    'current_time': current_time
} for trip in trips.itertuples()]




# Add stop markers to the map initially
def add_stop_markers(stops, map):
    stopNumber = 0 
    for stop in stops.itertuples():
        folium.Marker(
            location=[stop.stop_lat, stop.stop_lon],
            popup=f"Stop: {stop.stop_name}, Stop Number: {stopNumber}"
        ).add_to(map)
        stopNumber += 1
    return map

map = add_stop_markers(stops, map)

def create_lines(shapes, map):
    shape_locations = []
    
    for shape in shapes.itertuples():
        shape_locations.append([shape.shape_pt_lat, shape.shape_pt_lon])
    
    if len(shape_locations) > 1:
        folium.PolyLine(locations=shape_locations, color="red", dash_array='5', weight=2.5, opacity=1).add_to(map)
    return map

map = create_lines(shapes, map)

# Dictionary to store bus markers
bus_markers = {}


def update_bus_positions(buses, stop_times, current_time):
    updated_buses = []
    for bus in buses:
        trip_id = bus['trip_id']
        stop_times_for_trip = stop_times[stop_times['trip_id'] == trip_id]
        
        if not stop_times_for_trip.empty:
            current_stop_sequence = bus['current_stop_sequence']
            
            if current_stop_sequence < len(stop_times_for_trip):
                current_stop = stop_times_for_trip.iloc[current_stop_sequence]
                arrival_time = datetime.strptime(current_stop['arrival_time'], '%H:%M:%S')
                
                if current_time >= arrival_time:
                    bus['current_stop_sequence'] += 1
            
            if bus['current_stop_sequence'] < len(stop_times_for_trip):
                updated_buses.append(bus)
    return updated_buses



def visualize_buses(buses, stops, stop_times, map):
    for bus in buses:
        trip_id = bus['trip_id']
        current_stop_sequence = bus['current_stop_sequence']
        
        if current_stop_sequence > 0:
            stop_times_for_trip = stop_times[stop_times['trip_id'] == trip_id]
            
            if current_stop_sequence < len(stop_times_for_trip):
                current_stop_id = stop_times_for_trip.iloc[current_stop_sequence]['stop_id']
                stop = stops[stops['stop_id'] == current_stop_id]
                
                if not stop.empty:
                    folium.Marker(
                        location=[stop['stop_lat'].values[0], stop['stop_lon'].values[0]],
                        popup=f"Bus on route {bus['route_id']} at stop {stop['stop_name'].values[0]}"
                    ,icon=folium.Icon(color="red",icon="bus-simple", prefix='fa')).add_to(map)
    return map

# Simulation loop
while current_time <= end_time:
    buses = update_bus_positions(buses, stop_times, current_time)
    map = visualize_buses(buses, stops, stop_times, map)
    clear_output(wait=True)
    current_time += time_step
    display(map)
    # Adjust the sleep duration as needed
    time.sleep(30)
    
    

KeyboardInterrupt: 